# Writers Room Notebook: TV Screenplay Concept Generator + Feedback Loop

In [39]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
client = OpenAI()  # loads API key from env

In [40]:
import json
from pathlib import Path
from uuid import uuid4
from IPython.display import display, Markdown

In [41]:
# === Configuration ===
MEMORY_FILE = Path("writers_room_memory.json")

In [42]:
# === Memory Backend ===
def init_memory():
    if not MEMORY_FILE.exists():
        with open(MEMORY_FILE, "w") as f:
            json.dump({}, f)

def save_to_memory(table: str, key: str, content):
    init_memory()
    with open(MEMORY_FILE, "r+") as f:
        db = json.load(f)
        db.setdefault(table, {})[key] = content
        f.seek(0)
        json.dump(db, f, indent=2)
        f.truncate()

def get_from_memory(table: str, key: str):
    init_memory()
    with open(MEMORY_FILE, "r") as f:
        db = json.load(f)
    return db.get(table, {}).get(key, None)

In [43]:
def export_memory(file_path: str):
    init_memory()
    with open(MEMORY_FILE, "r") as src:
        with open(file_path, "w") as dst:
            dst.write(src.read())

def import_memory(file_path: str):
    with open(file_path, "r") as src:
        with open(MEMORY_FILE, "w") as dst:
            dst.write(src.read())

In [44]:
# === UI Helpers ===
def display_output(text):
    display(Markdown(f"### Output:\n{text}"))

def choose_from_versions(versions):
    for i, v in enumerate(versions):
        display(Markdown(f"**Version {i+1}:**\n{v}"))
    index = int(input("Choose final version (number): "))
    return versions[index - 1]

In [ ]:
def feedback_loop(component_name, generate_func, revise_func, prompt):
    versions = []
    feedback_history = []
    
    # Step 1: Generate initial output
    current_output = generate_func(prompt)
    versions.append(current_output)
    display_output(current_output)

    while True:
        feedback = input("\n Feedback on this version? Type to revise, or press Enter to accept: ").strip()

        if not feedback:
            # No feedback = accept the latest version
            break

        # Step 2: Revise based on feedback
        current_output = revise_func(versions[-1], feedback)
        versions.append(current_output)
        display_output(current_output)

    # Step 3: Choose final version from history
    final_version = choose_from_versions(versions)

    save_to_memory("concept", component_name, {
                   "final_thesis": final_version, 
                   "feedback_history": feedback_history
                   })
    
    print(f"\nFinal version {final_version} saved to memory under 'concept' → '{component_name}'")

In [46]:
def generate_show_thesis(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4", "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a TV writer brainstorming a show thesis."},
            {"role": "user", "content": f"Generate a compelling TV show thesis for this idea: {prompt}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()



def revise_with_feedback(previous_output, feedback):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a professional TV writer revising a show thesis based on feedback."},
            {"role": "user", "content": f"Here's the current thesis:\n\n{previous_output}\n\nPlease revise it based on this feedback:\n{feedback}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


In [47]:
# prompt = input("Enter your show idea prompt: ")
prompt = "A Gen Z drama set at an elite ski racing academy in Aspen, where ambition, class, and friendship collide on and off the slopes"

In [48]:
# # === Try It Out ===
# #prompt = input("Enter your show idea prompt: ")
# show_thesis = generate_show_thesis(prompt)
# display_output(show_thesis)

# memory_category = "concept"
# memory_title = "show_thesis"
# save_to_memory(memory_category, memory_title, show_thesis)
# print(f"Saved to memory under '{memory_category}' → '{memory_title}'")

In [50]:
# === Feedback Loop ===
final_thesis = feedback_loop(
    "show_thesis", 
    generate_show_thesis, 
    revise_with_feedback, 
    prompt)

### Output:
**Title: "Slalom"**

**Thesis:**

"Slalom" delves into the high-stakes world of an elite ski racing academy in Aspen, where the snow-covered peaks are as treacherous as the social dynamics that unfold beneath them. This Gen Z drama explores the lives of a diverse group of young athletes driven by ambition, as they navigate the slippery slopes of competition, privilege, and identity.

At the prestigious Aspen Alpine Academy, skiing is not just a sport—it's a way of life, a ticket to glory, and for some, a means to escape the shadows of their past. Against the backdrop of breathtaking winter landscapes, students from vastly different backgrounds clash and connect, united by their shared passion but divided by their personal ambitions and the invisible lines of class and privilege.

As the pressure to succeed mounts, friendships are tested, secrets are unearthed, and rivalries intensify both on the slopes and in the halls of the academy. The show captures the essence of Gen Z, highlighting their resilience and determination to forge their own paths while confronting societal expectations and their own inner conflicts.

"Slalom" weaves a narrative that is as much about the thrill of racing and the pursuit of excellence as it is about the bonds that form and fracture under pressure. It examines the complexities of growing up in a world where the stakes are high, and every decision could mean the difference between triumph and downfall. With its dynamic characters and intricate storytelling, "Slalom" offers a fresh, nuanced perspective on ambition, friendship, and the pursuit of dreams in a world where the only constant is change.

### Output:
**Title: "The Fall Line"**

**Thesis:**

"The Fall Line" immerses viewers in the high-stakes world of an elite ski racing academy in Aspen, where the majestic snow-covered peaks are as unpredictable as the social dynamics that unfold beneath them. This Gen Z drama follows a diverse ensemble of young athletes, driven by ambition, as they navigate the slippery slopes of competition, privilege, and identity.

At the prestigious Aspen Alpine Academy, skiing is more than just a sport—it's a lifestyle, a pathway to glory, and for some, an escape from the shadows of their past. Against the backdrop of breathtaking winter landscapes, students from diverse backgrounds clash and connect. They are united by their shared passion, yet divided by personal ambitions and the invisible lines of class and privilege.

As the pressure to succeed mounts, friendships are tested, secrets are unearthed, and rivalries intensify both on the slopes and within the halls of Aspen Alpine Academy. The show captures the essence of Gen Z, highlighting their resilience and determination to forge their own paths while confronting societal expectations and their own inner conflicts.

"The Fall Line" weaves a narrative that is as much about the thrill of racing and the pursuit of excellence as it is about the bonds that form and fracture under pressure. It examines the complexities of growing up in a world where the stakes are high, and every decision could mean the difference between triumph and downfall. With its dynamic characters and intricate storytelling, "The Fall Line" offers a fresh, nuanced perspective on ambition, friendship, and the pursuit of dreams in a world where the only constant is change.

### Output:
**Title: "The Fall Line"**

**Thesis:**

"The Fall Line" immerses viewers in the high-stakes world of an elite ski racing academy in Aspen, where the majestic snow-covered peaks mirror the unpredictable social dynamics unfolding beneath. This Gen Z drama follows four distinct young athletes, each driven by ambition and personal struggles, as they navigate the slippery slopes of competition, privilege, and identity.

At the prestigious Aspen Alpine Academy, skiing is more than just a sport—it's a lifestyle, a pathway to glory, and for some, an escape from the shadows of their past. The story is explored through the individual experiences and interconnected relationships of four main characters: Wes, Claire, Chase, and Tora.

Wes, the talented "townie," uses his mountain expertise to challenge the privileged elite, becoming a formidable presence, especially for Claire. As the heiress of a skiing dynasty, Claire feels the weight of family expectations and finds her world upended by Wes's undeniable skill. Her boyfriend, Chase, the academy's golden boy, is caught between maintaining his top status and battling his own ego. His rivalry with Wes becomes as much about proving himself as it is about winning. Meanwhile, Tora, the free spirit, keeps everyone guessing with her daring maneuvers and antics. Beneath her wild exterior lies a deep desire to be seen; her audacious stunts are both a bid for attention and an expression of her quest for freedom.

Against the backdrop of breathtaking winter landscapes, these students from diverse backgrounds clash and connect, united by their shared passion but divided by personal ambitions and the invisible lines of class and privilege. As the pressure to succeed mounts, friendships are tested, secrets are unearthed, and rivalries intensify both on the slopes and within the halls of Aspen Alpine Academy.

"The Fall Line" captures the essence of Gen Z, highlighting their resilience and determination to forge their own paths while confronting societal expectations and inner conflicts. It weaves a narrative as much about the thrill of racing and the pursuit of excellence as it is about the bonds that form and fracture under pressure. The show examines the complexities of growing up in a world where the stakes are high, and every decision could mean the difference between triumph and downfall. With its dynamic characters and intricate storytelling, "The Fall Line" offers a fresh, nuanced perspective on ambition, friendship, and the pursuit of dreams in a world where the only constant is change.

### Output:
**Title: "The Fall Line"**

**Revised Thesis:**

"The Fall Line" takes viewers deep into the exhilarating and treacherous world of an elite ski racing academy in Aspen, where the awe-inspiring snow-covered peaks serve as a metaphor for the tumultuous social dynamics beneath the surface. This Gen Z drama centers on themes of class, identity, belonging, coming of age, and ambition, following four diverse young athletes as they navigate the exhilarating yet perilous landscape of competition and self-discovery.

At the prestigious Aspen Alpine Academy, skiing is not just a sport—it's a way of life, a gateway to success, and for some, a refuge from their pasts. The narrative unfolds through the lives of four main characters: Wes, Claire, Chase, and Tora, each wrestling with their own aspirations and identities.

Wes, the skilled "townie," brings his mountain expertise to challenge the academy's privileged elite, disrupting the status quo and particularly impacting Claire. As the heiress of a renowned skiing dynasty, Claire grapples with her family's towering expectations and finds herself shaken by Wes's raw talent and conviction. Her boyfriend Chase, the academy's golden boy, is torn between maintaining his top ranking and confronting his own insecurities. His rivalry with Wes becomes a battleground for self-worth and validation. Meanwhile, Tora, the free spirit, defies conventions with her daring stunts and audacious behavior. Beneath her bold exterior lies a profound yearning for recognition and a quest for belonging.

Set against the dramatic backdrop of Aspen's winter landscapes, these students from varied backgrounds intersect and collide, united by their shared passion yet divided by personal ambitions and the unspoken boundaries of class and privilege. As the pressure to excel intensifies, friendships are tested, secrets unravel, and rivalries deepen both on the slopes and within the academy's walls.

"The Fall Line" captures the essence of Gen Z, portraying their resilience and determination to carve out their own identities while confronting societal pressures and personal dilemmas. It weaves a narrative that explores the thrilling pursuit of excellence and the intricate bonds that form and fracture under pressure. The show examines the complexities of growing up in a world where every decision is a step toward defining who they are and where they belong. With its dynamic characters and layered storytelling, "The Fall Line" offers a fresh, nuanced perspective on ambition, friendship, and the journey to find one's place in a constantly evolving world.

### Output:
**Title: "The Fall Line"**

**Revised Thesis:**

"The Fall Line" immerses viewers in the exhilarating and treacherous world of an elite ski racing academy in Aspen, where the awe-inspiring snow-covered peaks mirror the complex social dynamics beneath the surface. This Gen Z drama explores themes of class, identity, belonging, coming of age, and ambition, following four diverse young athletes as they navigate the thrilling yet perilous landscape of competition and self-discovery.

At the prestigious Aspen Alpine Academy, skiing transcends sport—it's a lifestyle, a pathway to success, and for some, an escape from their pasts. The story unfolds through the lives of four main characters: Wes, Claire, Chase, and Tora, each wrestling with their own aspirations and identities.

Wes, the easy-going "townie," is a natural athlete who is one with the mountain. He feels out of place in the academy's elite world and doesn't particularly like it, but his passion for skiing and racing drives him forward. His presence challenges the academy's privileged elite, particularly impacting Claire. An heiress of a renowned skiing dynasty, Claire grapples with her family's towering expectations and finds herself unsettled by Wes's raw talent and genuine love for the sport. Her boyfriend Chase, the academy's golden boy, is caught between maintaining his top ranking and confronting his insecurities. His rivalry with Wes becomes a battleground for self-worth and validation. Meanwhile, Tora, the free spirit, defies conventions with her daring stunts and audacious behavior. Beneath her bold exterior lies a profound yearning for recognition and a quest for belonging.

Set against Aspen's dramatic winter landscapes, these students from varied backgrounds intersect and collide, united by their shared passion yet divided by personal ambitions and the unspoken boundaries of class and privilege. As the pressure to excel intensifies, friendships are tested, secrets unravel, and rivalries deepen both on the slopes and within the academy's walls.

"The Fall Line" captures the essence of Gen Z, portraying their resilience and determination to carve out their identities while confronting societal pressures and personal dilemmas. It weaves a narrative that explores the thrilling pursuit of excellence and the intricate bonds that form and fracture under pressure. The show examines the complexities of growing up in a world where every decision is a step toward defining who they are and where they belong. With its dynamic characters and layered storytelling, "The Fall Line" offers a fresh, nuanced perspective on ambition, friendship, and the journey to find one's place in a constantly evolving world.

...

**Version 11:**
**Title: "The Fall Line"**

**Revised Thesis:**

"The Fall Line" thrusts viewers into the electrifying and emotionally charged world of Aspen Alpine Academy, an elite ski racing school where ambition, identity, and community converge on and off the slopes. Inspired by the spirit of "Friday Night Lights," this Gen Z drama explores the passions and struggles of four young athletes—Wes, Claire, Chase, and Tora—as they navigate intense competition and personal discovery amid Aspen's stunning yet challenging backdrop.

Wes, a gifted local skier with a humble background, finds himself at odds with the academy's privileged culture. His raw talent and dedication to the sport resonate with the community, drawing both admiration and envy. Claire, the heiress of a prestigious skiing family, wrestles with the weight of expectation while finding solace in Wes's genuine nature. Her boyfriend, Chase, the academy's star racer, battles internal demons as Wes's rising prowess threatens his dominance, igniting a fierce rivalry that echoes through the school's halls.

Tora, celebrated for her rebellious spirit and fearless stunts, yearns for recognition in a world that often sidelines her uniqueness. Her journey to forge her path becomes a rallying point for others who feel marginalized, reflecting the power of individuality within a community striving for excellence.

As these athletes pursue their dreams, they confront class divides, personal ambitions, and evolving relationships, all while the eyes of the Aspen community are upon them. "The Fall Line" captures the essence of Gen Z, echoing the heart of "Friday Night Lights" by showcasing the strength of community, the pursuit of authenticity, and the resilience required to thrive under pressure. With vibrant characters and dynamic storytelling, the show offers a fresh perspective on identity, friendship, and belonging in a rapidly changing world.

**Version 12:**
**Title: "The Fall Line"**

**Revised Thesis:**

"The Fall Line" immerses viewers in the dynamic and emotionally charged world of Aspen Alpine Academy, an elite ski racing school where ambition, identity, and community intersect on and off the slopes. This Gen Z drama explores the passions and struggles of four young athletes—Wes, Claire, Chase, and Tora—as they navigate intense competition and personal discovery amid Aspen's stunning yet challenging backdrop.

Wes, a talented local skier who grew up skiing powder, finds himself at odds with the academy's privileged culture. Though not antisocial, his humble background sets him apart from the elite world of the academy. His raw talent and dedication to the sport resonate with the community, drawing both admiration and envy. Claire, the heiress of a prestigious skiing family, struggles with the weight of expectation while finding solace in Wes's genuine nature. Her boyfriend, Chase, the academy's star racer, battles internal demons as Wes's rising prowess threatens his dominance, igniting a fierce rivalry that echoes through the school's halls.

Tora, celebrated for her rebellious spirit and fearless stunts, yearns for recognition in a world that often sidelines her uniqueness. Her journey to forge her path becomes a rallying point for others who feel marginalized, reflecting the power of individuality within a community striving for excellence.

As these athletes pursue their dreams, they confront class divides, personal ambitions, and evolving relationships, all while the eyes of the Aspen community are upon them. "The Fall Line" captures the essence of Gen Z by showcasing the strength of community, the pursuit of authenticity, and the resilience required to thrive under pressure. With vibrant characters and dynamic storytelling, the show offers a fresh perspective on identity, friendship, and belonging in a rapidly changing world.

**Version 13:**
**Revised Thesis:**

"The Fall Line" immerses viewers in the dynamic and emotionally charged world of Aspen Alpine Academy, an elite ski racing school where ambition, identity, and community intersect on and off the slopes. This Gen Z drama explores the passions and struggles of four young athletes—Wes, Claire, Chase, and Tora—as they navigate intense competition and personal discovery amid Aspen's stunning yet challenging backdrop.

Wes, a talented local skier who grew up skiing powder, finds himself at odds with the academy's privileged culture. Though not antisocial, his humble background sets him apart from the elite world of the academy. His raw talent and dedication to the sport resonate with the national race circuit, drawing both admiration and envy. Claire, the heiress of a prestigious skiing family, struggles with the weight of expectation while finding solace in Wes's genuine nature. Her boyfriend, Chase, the academy's star racer, battles internal demons as Wes's rising prowess threatens his dominance, igniting a fierce rivalry that echoes through the school's halls and beyond.

Tora, celebrated for her rebellious spirit and fearless stunts, yearns for recognition in a world that often sidelines her uniqueness. Her journey to forge her path becomes a rallying point for others who feel marginalized, reflecting the power of individuality within a community striving for excellence.

As these athletes pursue their dreams, they confront class divides, personal ambitions, and evolving relationships, all under the watchful eyes of their parents and the national race circuit. The pressure to succeed is amplified by the constant monitoring of scoreboards and academy social media, where every win and loss is scrutinized. "The Fall Line" captures the essence of Gen Z by showcasing the strength of community, the pursuit of authenticity, and the resilience required to thrive under pressure. With vibrant characters and dynamic storytelling, the show offers a fresh perspective on identity, friendship, and belonging in a rapidly changing world.

**Version 14:**
**Revised Thesis:**

"The Fall Line" plunges viewers into the world of Aspen Alpine Academy, a prestigious ski racing school where ambition, identity, and community collide. This Gen Z drama follows four young athletes—Wes, Claire, Chase, and Tora—as they navigate intense competition and personal growth amid Aspen's stunning backdrop.

Wes, a gifted local skier, feels out of place in the academy's elite culture but garners attention for his raw talent, sparking both admiration and rivalry. Claire, burdened by her family's legacy, finds comfort in Wes's authenticity, while her boyfriend, Chase, battles insecurities as his status is threatened. Tora, known for her rebellious spirit, seeks recognition in a world that often overlooks individuality.

Facing class divides and personal ambitions, these athletes strive for success under the scrutiny of their parents and the national race circuit. "The Fall Line" captures Gen Z's essence through themes of community, authenticity, and resilience, offering a fresh take on identity, friendship, and belonging in a rapidly evolving world.


Final version **Revised Thesis:**

"The Fall Line" plunges viewers into the world of Aspen Alpine Academy, a prestigious ski racing school where ambition, identity, and community collide. This Gen Z drama follows four young athletes—Wes, Claire, Chase, and Tora—as they navigate intense competition and personal growth amid Aspen's stunning backdrop.

Wes, a gifted local skier, feels out of place in the academy's elite culture but garners attention for his raw talent, sparking both admiration and rivalry. Claire, burdened by her family's legacy, finds comfort in Wes's authenticity, while her boyfriend, Chase, battles insecurities as his status is threatened. Tora, known for her rebellious spirit, seeks recognition in a world that often overlooks individuality.

Facing class divides and personal ambitions, these athletes strive for success under the scrutiny of their parents and the national race circuit. "The Fall Line" captures Gen Z's essence through themes of community, authenticity,

In [ ]:
# === Optional: Export/Import Memory ===
# export_memory("backup_writers_room.json")
# import_memory("backup_writers_room.json")